In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Constants
IMG_SIZE = 128
DATA_DIR = "full"
CATEGORIES = ["no_overflow", "overflow"]

# Load and preprocess dataset
def load_data(data_dir, categories):
    X, y = [], []
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        for img_file in os.listdir(path):
            try:
                img_path = os.path.join(path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img / 255.0)  # Normalize
                y.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(X), np.array(y)

X, y = load_data(DATA_DIR, CATEGORIES)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=len(CATEGORIES))
y_test = to_categorical(y_test, num_classes=len(CATEGORIES))

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')  # 2 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Save the model
model.save("garbage_overflow_model.h5")


Error loading image full\overflow\Garbage_bin_0001.xml: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image full\overflow\Garbage_bin_00010.xml: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image full\overflow\Garbage_bin_000100.xml: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image full\overflow\Garbage_bin_000101.xml: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image full\overflow\Garbage_bin_000102.xml: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\mo

c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 208ms/step - accuracy: 0.7046 - loss: 1.2490 - val_accuracy: 0.7941 - val_loss: 0.5256
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 194ms/step - accuracy: 0.7694 - loss: 0.5369 - val_accuracy: 0.8235 - val_loss: 0.4555
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - accuracy: 0.8437 - loss: 0.3962 - val_accuracy: 0.8588 - val_loss: 0.3220
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step - accuracy: 0.9005 - loss: 0.2287 - val_accuracy: 0.8588 - val_loss: 0.3227
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 193ms/step - accuracy: 0.9451 - loss: 0.1653 - val_accuracy: 0.9235 - val_loss: 0.1719
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 197ms/step - accuracy: 0.9574 - loss: 0.0848 - val_accuracy: 0.9294 - val_loss: 0.1704
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 196ms/step - accuracy: 0.9736 - loss: 0.0580 - val_accuracy: 0.9353 - val_loss: 0.1594
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 187ms/step - accuracy: 0.9819 - loss: 0.0437 - val_accuracy: 0.

In [3]:
def predict_overflow(image_path, model, categories):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    img = np.expand_dims(img, axis=0)
    
    prediction = model.predict(img)
    predicted_index = np.argmax(prediction)
    predicted_category = categories[predicted_index]
    
    if predicted_category == "overflow":
        send_alert()
    
    return predicted_category

def send_alert():
    print("⚠️ Alert: Garbage Overflow Detected! Take immediate action.")


In [8]:
# Load the trained model
from tensorflow.keras.models import load_model

model = load_model("garbage_overflow_model.h5")

# Test the model with a new image
image_path = "gbno.jpeg"
predicted_category = predict_overflow(image_path, model, CATEGORIES)

print(f"Predicted Category: {predicted_category}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Predicted Category: no_overflow


In [6]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9502 - loss: 0.1244
Test Accuracy: 94.71%
